In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from osgeo import osr, ogr
import os
os.environ['GDAL_DATA'] = 'C:\\Users\\Enzo\\Anaconda3\\pkgs\\libgdal-2.3.3-h10f50ba_0\\Library\\share\\gdal'
from geopandas import GeoSeries
from shapely.geometry import Polygon
from shapely.geometry import Point
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
##Model Classification
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df_target = pd.read_csv("C:\\Users\\Enzo\\Desktop\\hackathon\\TARGET-communities-extract-wellbeing.csv")
df_real_estate = pd.read_csv("C:\\Users\\Enzo\\Desktop\\hackathon\\UTSEUS-anjuke-real-estate.csv")

In [81]:
df_real_estate.head()

,id,address,longitude,latitude,bedroom,room,surface,price,onesquaremeter,tags,district,neighborhood,done
0,A888553302,宝山-大场 金地艺境(宝山)(555弄),121.343927,31.319356,2,2,30.0,1160000.0,38666.666667,金地开发 **首推 地铁首通 价格首降 配套就是你能想到的,baoshan,dachang,1
1,A888376847,宝山-大场 骏华苑,121.407325,31.302368,1,1,38.0,1950000.0,51315.789474,龙璟公馆嘉兴市近万达广场，北大附属实验校区，地铁沿线高铁站旁,baoshan,dachang,1
2,A885088482,宝山-大场 琉樽尚品(255弄),121.397487,31.290871,3,2,92.0,5350000.0,58152.173913,92+7平书房，婚房装修30万！满2年税少11万！再降20万,baoshan,dachang,1
3,A885392981,宝山-大场 琉樽尚品（255弄）,121.397487,31.290871,2,2,68.0,4350000.0,63970.588235,中环景观，电梯复式，豪装45万全送，拎包入住，小格局大设计，,baoshan,dachang,1
4,A885831305,宝山-大场 美好家小区,121.421397,31.302658,2,2,80.0,4000000.0,50000.000000,美好家南北通精装2房板式房型采光好随时好看带天井,baoshan,dachang,1


In [3]:
##Fonction to transform longitude and Latitude to metrics position

old_crs = osr.SpatialReference()
old_crs.ImportFromEPSG(4326)
new_crs = osr.SpatialReference()
new_crs.ImportFromEPSG(4479)
transform = osr.CoordinateTransformation(old_crs,new_crs)

def getNewLongitude(xy):
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(xy[0], xy[1])
    point.Transform(transform)
    return point.GetX()

def getNewLatitude(xy):
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(xy[0], xy[1])
    point.Transform(transform)
    return point.GetY()

In [4]:
#Transforming to metrics
df_target['start_lon'] = df_target[['longitude', 'latitude']].apply(getNewLongitude, axis=1)
df_target['start_lat'] = df_target[['longitude', 'latitude']].apply(getNewLatitude, axis=1)

df_target.head()

,Unnamed: 0,clean,smell,noise,town,district,longitude,latitude,start_lon,start_lat
0,万寿社区居委会,3,0.0,0.0,盈浦街道,青浦区,121.104287,31.155882,-2.822208e+06,4.677636e+06
1,万泰花园第一居委会,3,1.0,0.0,七宝镇,闵行区,121.351529,31.144327,-2.842711e+06,4.665980e+06
2,万科阳光苑居委会,2,0.0,0.0,吴泾镇,闵行区,121.468520,31.042285,-2.855282e+06,4.665149e+06
3,万豪居委会,3,0.0,0.0,长征镇,普陀区,121.367237,31.240725,-2.841109e+06,4.660474e+06
4,万里名轩社区居委会,2,NaN,0.0,万里街道,普陀区,121.403601,31.269155,-2.843214e+06,4.657274e+06


In [940]:
#Transforming to metrics
df_real_estate['start_lon'] = df_real_estate[['longitude', 'latitude']].apply(getNewLongitude, axis=1)
df_real_estate['start_lat'] = df_real_estate[['longitude', 'latitude']].apply(getNewLatitude, axis=1)

In [5]:
## Cleaned/Processed CSV From US01 Session
df_mobike=pd.read_csv('mobike.csv')

In [6]:
#Sampling to fasten the processing
df_mobike = df_mobike.sample(n=20000)

In [941]:
#Creating Geopandas files
gdf_target=gpd.GeoDataFrame(df_target, geometry=[Point(x,y) for x,y in zip(df_target.start_lon, df_target.start_lat)])
gdf_real_estate=gpd.GeoDataFrame(df_real_estate, geometry=[Point(x,y) for x,y in zip(df_real_estate.start_lon, df_real_estate.start_lat)])
gdf_target.head()

,Unnamed: 0,clean,smell,noise,town,district,longitude,latitude,start_lon,start_lat,geometry
0,万寿社区居委会,3,0.0,0.0,盈浦街道,青浦区,121.104287,31.155882,-2.822208e+06,4.677636e+06,POINT (-2822207.586544672 4677635.626542749)
1,万泰花园第一居委会,3,1.0,0.0,七宝镇,闵行区,121.351529,31.144327,-2.842711e+06,4.665980e+06,POINT (-2842710.938459042 4665979.72604281)
2,万科阳光苑居委会,2,0.0,0.0,吴泾镇,闵行区,121.468520,31.042285,-2.855282e+06,4.665149e+06,POINT (-2855282.42476444 4665148.86376532)
3,万豪居委会,3,0.0,0.0,长征镇,普陀区,121.367237,31.240725,-2.841109e+06,4.660474e+06,POINT (-2841108.7489435 4660473.749169292)
4,万里名轩社区居委会,2,NaN,0.0,万里街道,普陀区,121.403601,31.269155,-2.843214e+06,4.657274e+06,POINT (-2843213.85608785 4657273.732676618)


In [9]:
gdf_mobike=gpd.GeoDataFrame(df_mobike, geometry=[Point(x,y) for x,y in zip(df_mobike.start_lon, df_mobike.start_lat)])
gdf_mobike = gdf_mobike.reset_index()

In [ ]:
##Creating "Bubbles" of 500m radius around our output datas and find which real estate data are inside those "bubbles"
data = pd.DataFrame()
for i,geo_target in gdf_target['geometry'].items():
    data[i]=gdf_real_estate.within(geo_target.buffer(500))
    print(i)
data.head()

0
1


In [ ]:
##Saving the data
data.to_csv('data_in_mobike.csv', index=0)

In [ ]:
## Creating the dataframe to put mobike data
in_data_mobike = pd.DataFrame(index=range(0,640), columns=['n_mobike'])

In [15]:
## Getting the amount of moobikes inside each 500m "Bubbles"
for i,geo_target in gdf_target['geometry'].items():
    in_data_mobike['n_mobike'][i]=0
    for j, geo in gdf_mobike['geometry'].items():
        if geo.within(geo_target.buffer(500)):
            in_data_mobike['n_mobike'][i]+=1
    print(i)

in_data_mobike.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,n_mobike
0,0
1,13
2,1
3,71
4,85


In [16]:
## Saving the data
in_data_mobike.to_csv('data_in_mobike.csv', index=0)

In [12]:
data= pd.read_csv('data_real_estate.csv')

In [48]:
## Creating the dataframe to put real estate data
df_target['price']= pd.Series()
df_target['meter']= pd.Series()
df_target['surface']= pd.Series()
df_target.head()

,Unnamed: 0,clean,smell,noise,town,district,longitude,latitude,start_lon,start_lat,geometry,price,meter,surface
0,万寿社区居委会,3,0.0,0.0,盈浦街道,青浦区,121.104287,31.155882,-2.822208e+06,4.677636e+06,POINT (-2822207.586544672 4677635.626542749),NaN,NaN,NaN
1,万泰花园第一居委会,3,1.0,0.0,七宝镇,闵行区,121.351529,31.144327,-2.842711e+06,4.665980e+06,POINT (-2842710.938459042 4665979.72604281),NaN,NaN,NaN
2,万科阳光苑居委会,2,0.0,0.0,吴泾镇,闵行区,121.468520,31.042285,-2.855282e+06,4.665149e+06,POINT (-2855282.42476444 4665148.86376532),NaN,NaN,NaN
3,万豪居委会,3,0.0,0.0,长征镇,普陀区,121.367237,31.240725,-2.841109e+06,4.660474e+06,POINT (-2841108.7489435 4660473.749169292),NaN,NaN,NaN
4,万里名轩社区居委会,2,NaN,0.0,万里街道,普陀区,121.403601,31.269155,-2.843214e+06,4.657274e+06,POINT (-2843213.85608785 4657273.732676618),NaN,NaN,NaN


In [ ]:
## Getting the median Price; price/squaremeters and surface inside every bubbles
columns = ['price','meter', 'surface']
in_data = pd.DataFrame(index=range(0,640), columns=columns)
for i in range(0, len(gdf_target)):
    mean = pd.DataFrame(index=range(0,1000), columns=columns)
    print(i)
    k=0
    for j in range(0, len(data)):
        if data[i][j]==True:
            mean['price'][k] = gdf_real_estate['price'][j]
            mean['meter'][k] = gdf_real_estate['onesquaremeter'][j]
            mean['surface'][k] = gdf_real_estate['surface'][j]
            k=k+1
    if len(mean[mean['price']>=0]['price'])>0:
        in_data['price'][i]=np.median(mean[mean['price']>=0]['price'])
        in_data['meter'][i]=np.median(mean[mean['meter']>=0]['meter'])
        in_data['surface'][i]=np.median(mean[mean['surface']>=0]['surface'])
## Print index for which there are no real estate data in the area
    else :
        print('no data for',i)


        

In [82]:
in_data.to_csv('data_in_2.csv', index=0)

In [17]:
in_data = pd.read_csv('data_in.csv')
in_data.head()

,price,meter,surface
0,1860000.0,31578.947368,58.0
1,5800000.0,58546.986049,96.0
2,3200000.0,42696.629213,79.0
3,6300000.0,64347.826087,93.0
4,6800000.0,67346.938776,103.0


In [951]:
## Create in and out datasets for the algorithm and taking out index with no data
X = in_data.drop({19,39,43,140,173,271,276,297,312,320,360,369,374,398,405,414,426})[['price', 'meter', 'surface']]
X['n_mobike']=in_data_mobike.drop({19,39,43,140,173,271,276,297,312,320,360,369,374,398,405,414,426})['n_mobike']
Y = df_target.drop({19,39,43,140,173,271,276,297,312,320,360,369,374,398,405,414,426,460})['clean']
X=X.reset_index(drop=True)
Y=Y.reset_index(drop=True)

In [948]:
## Scale the input data
X = MinMaxScaler().fit_transform(X)

In [949]:
X

array([[0.04052443, 0.17638841, 0.12177122, 0.        ],
       [0.08685936, 0.30761825, 0.15498155, 0.04626335],
       [0.10309893, 0.26758595, 0.2398524 , 0.00355872],
       ...,
       [       nan,        nan,        nan,        nan],
       [       nan,        nan,        nan,        nan],
       [       nan,        nan,        nan,        nan]])

In [651]:
## Creating sample with for train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, Z_train, Z_test = train_test_split(X, Z, test_size=0.5)

In [ ]:
## Testing the Model
clf = KNeighborsClassifier(weights='distance')
scores = cross_val_score(clf, X_train, Y_train, cv=ShuffleSplit(n_splits=20))
np.mean(scores)

In [938]:
## Predicting the Data test and asses accuracy
clf = KNeighborsClassifier(weights='distance')
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
np.mean(Y_pred == Y_test)

0.509009009009009

In [939]:
## maxtrix of results/prediction
conf_matrix = np.zeros((4,4))
for k in range(len(Y_pred)):
    conf_matrix[int(Y_pred[k])-1, int(Y_test.reset_index(drop=True)[k])-1]+=1
print(conf_matrix)

[[ 0.  0.  0.  0.]
 [ 5. 29. 35.  2.]
 [ 4. 59. 81.  1.]
 [ 1.  1.  1.  3.]]


In [877]:
## weighted error
error=0
for i in range (0,4):
    for j in range(0,4):
        error+=abs(i-j)* conf_matrix[i,j]
error

142.0

In [ ]:
error = error/len(Z_pred)